In [2]:
import os

print(f"进程{os.getpid()}开始")
pid = os.fork()  # 返回2次，父进程：返回子进程的ID，子进程：永远返回0

if pid == 0:
    print(f"我是子进程{os.getpid()}，我的父进程是{os.getppid()}")
else:
    print(f"我{os.getpid()}只是创建了一个子进程{pid}")

进程94723开始
我94723只是创建了一个子进程94750
我是子进程94750，我的父进程是94723


In [1]:
from multiprocessing import Process
import os


def run_proc(name):
    print(f"运行子进程{name}({os.getpid()})")


if __name__ == "__main__":
    print(f"父进程是{os.getpid()}")
    p = Process(target=run_proc, args=("test",))
    print("子进程即将开始")
    p.start()
    p.join()  # 等待子进程结束并继续运行
    print("子进程结束")

父进程是10228
子进程即将开始
运行子进程test(10233)
子进程结束


## Pool
#### 如果要启动大量的子进程，可以用进程池的方式批量创建子进程：


In [10]:
from multiprocessing import Pool
import os, time, random


def long_time_task(name):
    print(f"运行任务{name}({os.getpid()})")
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print(f"任务{name}运行了{end-start}秒")


if __name__ == "__main__":
    print(f"父进程{os.getpid()}.")
    p = Pool(2)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print("等待所有子进程结束...")
    p.close()
    p.join()
    print("所有子进程已经结束")

父进程85469.
运行任务0(93594)
运行任务1(93595)
等待所有子进程结束...
任务0运行了1.176269769668579秒
运行任务2(93594)
任务1运行了2.551132917404175秒
运行任务3(93595)
任务2运行了1.9671962261199951秒
运行任务4(93594)
任务3运行了1.2021350860595703秒
任务4运行了2.0198662281036377秒
所有子进程已经结束


## 子进程
#### 很多时候，子进程并不是自身，而是一个外部进程。我们创建了子进程后，还需要控制子进程的输入和输出。
#### subprocess模块可以让我们非常方便地启动一个子进程，然后控制其输入和输出。
#### 下面的例子演示了如何在Python代码中运行命令nslookup www.python.org，这和命令行直接运行的效果是一样的：

In [12]:
import subprocess

print("$ nslookup www.python.org")
r = subprocess.call(["nslookup", "www.python.org"])
print("退出代码:", r)

$ nslookup www.python.org
退出代码: 0


In [14]:
import subprocess

print("$ nslookup")
p = subprocess.Popen(
    ["nslookup"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
output, err = p.communicate(b"set q=mx\npython.org\nexit\n")  # 手动输入
print(output.decode("utf-8"))
print("Exit code:", p.returncode)

$ nslookup
Server:		8.8.8.8
Address:	8.8.8.8#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


## 进程间通信
#### Process之间肯定是需要通信的，操作系统提供了很多机制来实现进程间的通信。Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。
#### 我们以Queue为例，在父进程中创建两个子进程，一个往Queue里写数据，一个从Queue里读数据：

In [15]:
from multiprocessing import Process, Queue
import os, time, random


def write(q):
    print(f"处理写的进程:{os.getpid()}")
    for value in ["A", "B", "C"]:
        print(f"将{value}压入队列...")
        q.put(value)
        time.sleep(random.random())


def read(q):
    print(f"处理读取的进程:{os.getpid()}")
    while True:
        value = q.get(True)
        print(f"从队列中获取{value}")


if __name__ == "__main__":
    # 父进程创建Queue,并传给各个子进程
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))

    # 启动子进程pw,写入:
    pw.start()

    # 启动子进程pr,读取:
    pr.start()

    # 等待pw结束
    pw.join()

    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

处理写的进程:44711
将A压入队列...
处理读取的进程:44712
从队列中获取A
将B压入队列...
从队列中获取B
将C压入队列...
从队列中获取C


## 小结
#### 在Unix/Linux下，可以使用fork()调用实现多进程。

#### 要实现跨平台的多进程，可以使用multiprocessing模块。

#### 进程间通信是通过Queue、Pipes等实现的。

In [17]:
from multiprocessing import Process
import time


def action(a):  # 待会两个进程要执行的任务↓
    for i in range(30):  # 循环30次
        print(a, " ", i)
        time.sleep(0.1)  # 等待0.1s


if __name__ == "__main__":  # 这行代码很重要，新建进程的时候都加上它！！原因不用管（我也不知道233）

    jc1 = Process(target=action, args=("进程一",))
    jc2 = Process(target=action, args=("进程二",))

    jc1.start()  # 将蓄势待发的jc1进程正式启动！！
    jc2.start()  # 同上...

    jc1.join()  # 等待进程jc1将任务执行完...
    jc2.join()  # ...
    print("jc1,jc2任务都已执行完毕")

    jc1.close()  # 彻底关闭进程jc1
    jc2.close()  # ...

进程一   0
进程二   0
进程一   1
进程二   1
进程一   2
进程二   2
进程一   3
进程二   3
进程一   4
进程二   4
进程一   5
进程二   5
进程一   6
进程二   6
进程一   7
进程二   7
进程一   8
进程二   8
进程一   9
进程二   9
进程一   10
进程二   10
进程一   11
进程二   11
进程一   12
进程二   12
进程一   13
进程二   13
进程一   14
进程二   14
进程一   15
进程二   15
进程一   16
进程二   16
进程一   17
进程二   17
进程一   18
进程二   18
进程一   19
进程二   19
进程一   20
进程二   20
进程一   21
进程二   21
进程一   22
进程二   22
进程一   23
进程二   23
进程一   24
进程二   24
进程一   25
进程二   25
进程一   26
进程二   26
进程一   27
进程二   27
进程一   28
进程二   28
进程一   29
进程二   29
jc1,jc2任务都已执行完毕


In [18]:
# 进程池批量创建子进程
from multiprocessing import Pool
import time
import os


def action1(a, b=50):
    for i in range(b):
        print(a, "-->", os.getpid(), " ", i)
        time.sleep(0.1)


if __name__ == "__main__":  # 还要添加这行，否则可能出现异常

    ci = Pool(3)  # 创建一个进程池，容量为3个进程
    ci.apply_async(action1, args=("进程一",))  # 启动第一个子进程...
    ci.apply_async(action1, args=("进程二",))  # 和普通进程的启动方式有很大不同仔细看
    ci.apply_async(action1, args=("进程三", 60))  # Pool的最基本格式记住←
    # 注意：程序现在有4个进程在运行：上面的三个子进程 和一个最为核心的：主进程

    ci.close()  # 关闭进程池（但池子内已启动的子进程还会继续进行）
    ci.join()  # 等待进程池内的所有子进程完毕
    print("比如说这最后的一行输出就是主进程执行任务打印出来的")

##主进程（父进程）全程干了什么？创建进程池、启动子进程、关闭进程池、等待子进程完毕、打印最后一行

进程一 --> 55896   0
进程二 --> 55897   0
进程三 --> 55898   0
进程一 --> 55896   1
进程三 --> 55898   1
进程二 --> 55897   1
进程一 --> 55896   2
进程二 --> 55897   2
进程三 --> 55898   2
进程二 --> 55897   3
进程三 --> 55898   3
进程一 --> 55896   3
进程三 --> 55898   4
进程二 --> 55897   4
进程一 --> 55896   4
进程一 --> 55896   5
进程二 --> 55897   5
进程三 --> 55898   5
进程一 --> 55896   6
进程二 --> 55897   6
进程三 --> 55898   6
进程三 --> 55898   7
进程一 --> 55896   7
进程二 --> 55897   7
进程三 --> 55898   8
进程一 --> 55896   8
进程二 --> 55897   8
进程一 --> 55896   9
进程二 --> 55897   9
进程三 --> 55898   9
进程二 --> 55897   10
进程一 --> 55896   10
进程三 --> 55898   10
进程一 --> 55896   11
进程三 --> 55898   11
进程二 --> 55897   11
进程三 --> 55898   12
进程一 --> 55896   12
进程二 --> 55897   12
进程三 --> 55898   13
进程二 --> 55897   13
进程一 --> 55896   13
进程一 --> 55896   14
进程三 --> 55898   14
进程二 --> 55897   14
进程二 --> 55897   15
进程一 --> 55896   15
进程三 --> 55898   15
进程一 --> 55896   16
进程三 --> 55898   16
进程二 --> 55897   16
进程二 --> 55897   17
进程一 --> 55896   17
进程三 --> 55898   17
进程三 

In [20]:
# 进程间的通信
from multiprocessing import Queue, Process


def foo(queue):
    ss = queue.get()  # 管子的另一端放在子进程这里，子进程接收到了数据
    print("子进程已收到数据...")
    print(ss)  # 子进程打印出了数据内容...


if __name__ == "__main__":
    queue = Queue()  # 创建进程通信的Queue，你可以理解为我拿了个管子来...
    p = Process(target=foo, args=(queue,))  # 创建子进程

    print("主进程准备发送数据...")
    queue.put("有内鬼，终止交易！")  # 将管子的一端放在主进程这里，主进程往管子里丢入数据↑
    p.start()  # 启动子进程

    p.join()

主进程准备发送数据...
子进程已收到数据...
有内鬼，终止交易！
